In [10]:
import pandas as pd

# Load the uploaded CSV file
file_path = "D:\internship\customer_segmentation\data\customer_transactions_mock_data.csv"
df = pd.read_csv(file_path)




In [11]:
# Display the first few rows of the DataFrame
df.head()

,TransactionID,CustomerID,TransactionDate,ProductID,Quantity,UnitPrice,TotalPrice
0,TRX50000_0,CUST1302,2022-01-12,Prod_107,3,53.74,161.22
1,TRX50000_1,CUST1302,2022-01-12,Prod_104,2,43.15,86.30
2,TRX50000_2,CUST1302,2022-01-12,Prod_107,1,59.80,59.80
3,TRX50001_0,CUST1267,2023-10-17,Prod_108,1,214.31,214.31
4,TRX50001_1,CUST1267,2023-10-17,Prod_109,3,312.02,936.06


In [12]:
# Display basic information 
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15143 entries, 0 to 15142
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    15143 non-null  object 
 1   CustomerID       15143 non-null  object 
 2   TransactionDate  15143 non-null  object 
 3   ProductID        15143 non-null  object 
 4   Quantity         15143 non-null  int64  
 5   UnitPrice        15143 non-null  float64
 6   TotalPrice       15143 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 828.3+ KB


📊 Data Overview
Rows: 15,143 transactions

Columns: 7

TransactionID: Unique ID for each transaction

CustomerID: Unique identifier for customers

TransactionDate: Date of transaction (currently string)

ProductID: Product purchased

Quantity: Number of units bought

UnitPrice: Price per unit

TotalPrice: Quantity × UnitPrice



✅ Next Step: Data Preparation
We'll now:

Convert TransactionDate to datetime.

Check and handle any missing values or anomalies.

Inspect data types and value distributions.


In [5]:
# Convert 'TransactionDate' to datetime
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

In [7]:
# Check for missing values  
df.isnull().sum()

TransactionID      0
CustomerID         0
TransactionDate    0
ProductID          0
Quantity           0
UnitPrice          0
TotalPrice         0
dtype: int64

In [9]:
# basic statistics for anomaly detection
df.describe()

,TransactionDate,Quantity,UnitPrice,TotalPrice
count,15143,15143.000000,15143.000000,15143.000000
mean,2022-12-28 14:33:25.996169984,1.998349,103.732894,207.328918
min,2022-01-01 00:00:00,1.000000,1.220000,1.540000
25%,2022-06-27 12:00:00,1.000000,34.010000,58.740000
50%,2022-12-28 00:00:00,2.000000,76.970000,137.970000
75%,2023-07-01 00:00:00,3.000000,154.180000,285.340000
max,2023-12-30 00:00:00,3.000000,393.870000,1181.610000
std,NaN,0.818851,85.986186,203.743701


In [ ]:

# Check for duplicate transactions
duplicates = df.duplicated().sum()
# Display the number of duplicate transactions
print(f"Number of duplicate transactions: {duplicates}")
# Remove duplicate transactions if any
if duplicates > 0:
    df = df.drop_duplicates()
    print("Duplicate transactions removed.")

Number of duplicate transactions: 0


🧹 Data Preparation Summary
✅ No missing values — all columns are complete.

📅 Date conversion successful — TransactionDate is now in datetime format.

📈 Basic Stats Check:

Quantity ranges from 1 to 3.

UnitPrice ranges from $1.22 to $393.87, and TotalPrice from $1.54 to $1,181.61 — no obvious outliers for small-scale retail.

✅ Next Step: RFM Feature Engineering
We'll calculate:

Recency = Days since last purchase (relative to a reference date)

Frequency = Number of purchases

Monetary = Total spending

In [4]:
# Reference date: 1 day after the latest transaction
reference_date = df['TransactionDate'].max() + pd.Timedelta(days=1)

# Group by CustomerID to compute RFM
rfm = df.groupby('CustomerID').agg({
    'TransactionDate': lambda x: (reference_date - x.max()).days,  # Recency
    'TransactionID': 'nunique',                                     # Frequency
    'TotalPrice': 'sum'                                             # Monetary
}).reset_index()

# Rename columns
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

rfm.head()


,CustomerID,Recency,Frequency,Monetary
0,CUST1000,9,41,6855.79
1,CUST1001,49,10,2856.43
2,CUST1002,7,25,2175.73
3,CUST1003,22,33,1085.57
4,CUST1004,134,27,6887.81


- Recency: Smaller is better (more recent)

- Frequency: Higher = more loyal

- Monetary: Higher = higher value

🔍 Next Step: Exploratory Analysis & Scaling
We’ll:

Plot distributions of R, F, M (check skewness)

Log-transform if needed

Scale features for clustering